# (5.1) Solve AFAPE for dataset completed with multiple imputation
Use an estimator to evaluate E[C|do(R_bar = 1)]. Also give valid confidence intervals (through estimating mean and variance). 

In [1]:
%load_ext autoreload
%autoreload 2

## Define paths

Paths for data

In [2]:
from afa.configurations.utils_static import specify_default_paths_static
# which dataset to work on 
dataset_name   = "synthetic_1"

# name for of missingness scenario 
miss_scenario  = 'MCAR_1'

# automatically specify some path locations (change paths manually if needed) 
paths = specify_default_paths_static(dataset_name = dataset_name , miss_scenario = miss_scenario) 

# name for agent 
agent_name            = 'DQN'
agent_dir = paths['data_dir'] + 'afa_agents' + '/' + agent_name + '/'

# how to name the afa_dataset
mi_model_name   =  'mi_simple'
mi_model_dir  =  paths['data_dir'] + 'mi_models/' + mi_model_name + '/'
afa_dataset_name = mi_model_name 

## Load afa dataset

In [3]:
from afa.data_modelling.datasets.data_loader.data_loader_static import DataLoader_static
from afa.data_modelling.missingness.multiple_imputation.multiple_imputed_data_loader.multiple_imputed_data_loader_static import MultipleImputedDataLoader_static
from afa.afa_datasets.afa_data_loader.afa_data_loader_static import AFADataLoader_static

2023-02-27 16:49:15.770346: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 16:49:15.898295: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-27 16:49:15.898349: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-27 16:49:16.555553: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
# load dataset 
data_loader = DataLoader_static( data_file                  = paths['data_file'],
                                 superfeature_mapping_file  = paths['superfeature_mapping_file'],
                                 problem_file               = paths['problem_file'],
                                 afa_problem_files          = paths['afa_problem_files'], 
                                 folds_file                 = paths['folds_file'] )
dataset = data_loader.load() 

In [5]:
# load mi dataset
augmented_data_file = mi_model_dir + 'results.hkl'
mi_model_params = None
mi_data_loader = MultipleImputedDataLoader_static(                   
                    augmented_data_file = augmented_data_file,
                    dataset             = dataset,
                    model_params        = mi_model_params) 
mi_dataset = mi_data_loader.load() 

In [6]:
#load afa_dataset
augmented_data_file = agent_dir + afa_dataset_name + '_' + 'results.hkl'
afa_agent_params = None
afa_data_loader = AFADataLoader_static(                   
                    augmented_data_file = augmented_data_file,
                    dataset             = mi_dataset,
                    model_params        = afa_agent_params) 
afa_dataset = afa_data_loader.load() 

2023-02-27 16:49:22.192272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-27 16:49:22.192418: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-27 16:49:22.192486: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-27 16:49:22.192545: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-27 16:49:22.192602: W tensorf

## Compute estimates 

In [24]:
J_bootstraps = afa_dataset.estimate_counterfactual_cost_mi(  mi_model_name  = mi_model_name , 
                                                             fold = 0, split = "val", 
                                                             n_samples = None, 
                                                             n_bootstraps = 10)

> /mnt/c/Users/henrik.vonkleist/Nextcloud/PhD/Code/Active Feature Acquisition/afa_ts/afa/afa_datasets/afa_dataset_static.py(274)estimate_counterfactual_cost_mi()
    272 
    273         # compute bootstraps
--> 274         J_bootstraps  = {}
    275         J_bootstraps[mi_model_name ] =np.random.normal(loc=J, scale=np.sqrt(Var), size=n_bootstraps)
    276 



ipdb>  exit


In [18]:
J_bootstraps 

{'mi_simple': array([1.86039031, 2.25157695, 1.44828481, 1.93997693, 2.11311716,
        1.67076759, 1.9676605 , 2.29518835, 1.06887482, 1.65833719])}

In [19]:
# save estimate
from afa.afa_models.afa_estimators.utils import save_results_bootstrapping
save_results_bootstrapping( J_bootstraps , agent_dir, afa_dataset_name = afa_dataset_name )

## Compute estimates for convergence
If we know the ground truth, we might be interesting in plotting convergence, for this we might want to compute estimates J for different amount of available datapoints. 

In [20]:
from afa.afa_models.afa_estimators.utils_static import define_afa_estimator_static
from afa.afa_models.afa_estimators.utils import compute_counterfactual_cost_convergence

In [21]:
# init estiamtor 
estimators = [define_afa_estimator_static(    estimator_name   = mi_model_name ,
                                              estimator_type   = 'simple_blocking' ,
                                              estimator_params = None) ]

In [22]:
# define estimators 
J_bootstraps_convergence, convergence_steps  = compute_counterfactual_cost_convergence(  afa_dataset = afa_dataset, 
                                                                                         estimators = estimators, 
                                                                                         fold = 0, split = "val", 
                                                                                         n_samples = 1, 
                                                                                         n_bootstraps = 10)

Estimate counterfactual average cost
  - x-axis (number of datapoints) =  [ 10  53 280]
  - number of bootstraps for estimation: 10


In [23]:
# save estimators
save_results_bootstrapping( J_bootstraps_convergence , agent_dir, convergence_steps = convergence_steps, afa_dataset_name = afa_dataset_name)